**Enunciado:** Realizar una herramienta ETL que permita obtener el datamart ventas por internet (Internet Sales) y ventas por revendedores (Reseller Sales) a partir de la base de datos operacional. Documentar las dimensiones que participan y las tablas de hechos (50%). (Octubre 22)

* ¿Cuáles son las dimensiones que participan?

* ¿Cuál es el nivel de granularidad de los datos?

* ¿Cuáles son los indicadores (Medidas) de cada tabla de hechos?

In [16]:
import pandas as pd
from sqlalchemy import create_engine, inspect
import yaml
import os
import numpy as np

In [2]:
config_path = os.path.join(os.getcwd(), "config.yml")

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)
    config_oltp = config['OLTP']
    config_olap = config['OLAP']

url_oltp = (f"mssql+pyodbc://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']},{config_oltp['port']}/{config_oltp['dbname']}"
          f"?driver={config_oltp['drivername'].replace(' ', '+')}")

url_olap = (f"mssql+pyodbc://{config_olap['user']}:{config_olap['password']}@{config_olap['host']},{config_olap['port']}/{config_olap['dbname']}"
           f"?driver={config_olap['drivername'].replace(' ', '+')}")
oltp = create_engine(url_oltp)
olap = create_engine(url_olap)

Leer las tablas

In [3]:
def cargaSegura(engine, schema, table):
    inspector = inspect(engine)

    # Obtener columnas
    columnas = [col["name"] for col in inspector.get_columns(table, schema=schema)]
    columnas_problematicas = []

    # Intentar cargar tabla completa
    try:
        return pd.read_sql_table(table_name=table, con=engine, schema=schema)
    except Exception:
        pass

    # Detectar columnas problemáticas
    for col in columnas:
        try:
            pd.read_sql_query(
                f'SELECT TOP 10 "{col}" FROM "{schema}"."{table}"',
                con=engine
            )
        except Exception:
            columnas_problematicas.append(col)


    # Columnas buenas
    columnas_ok = [col for col in columnas if col not in columnas_problematicas]

    # Si no hay columnas válidas
    if not columnas_ok:
        print(f"⚠ La tabla {schema}.{table} no tiene columnas convertibles. Retornando dataframe vacío.")
        return pd.DataFrame()

    # Cargar solo columnas válidas
    query = (
        f'SELECT {", ".join([f"""\"{c}\"""" for c in columnas_ok])} '
        f'FROM "{schema}"."{table}"'
    )

    df = pd.read_sql_query(query, con=engine)
    return df


def extractHumanResources(conection):
    tablas = [
        "Shift", "Department", "Employee", "EmployeeDepartmentHistory", "EmployeePayHistory"
    ]
    humanResources = {}
    for tabla in tablas:
        df = cargaSegura(conection, "HumanResources", tabla)
        humanResources[tabla] = df
        
    return humanResources

def extractPerson(conection):
    tablas = [
        "PersonPhone", "PhoneNumberType", "Address", "AddressType",
        "StateProvince", "BusinessEntity", "BusinessEntityAddress", "BusinessEntityContact",
        "ContactType", "CountryRegion", "EmailAddress", "Password", "Person"
    ]
    person = {}
    for tabla in tablas:
        df = cargaSegura(conection, "Person", tabla)
        person[tabla] = df
        
    return person

def extractProduction(conection):
    tablas = [
        "Product", "ScrapReason", "ProductCategory", "ProductCostHistory", "ProductDescription",
        "ProductDocument", "ProductInventory", "ProductListPriceHistory", "ProductModel",
        "ProductModelIllustration", "ProductModelProductDescriptionCulture", "BillOfMaterials",
        "ProductPhoto", "ProductProductPhoto", "TransactionHistory", "ProductReview",
        "TransactionHistoryArchive", "ProductSubcategory", "UnitMeasure", "WorkOrder",
        "Culture", "WorkOrderRouting", "Document", "Illustration", "Location"
    ]
    production = {}
    for tabla in tablas:
        df = cargaSegura(conection, "Production", tabla)
        production[tabla] = df
        
    return production

def extractPurchasing(conection):
    tablas = [
        "ShipMethod", "ProductVendor", "Vendor", "PurchaseOrderDetail", "PurchaseOrderHeader"
    ]
    purchasing = {}
    for tabla in tablas:
        df = cargaSegura(conection, "Purchasing", tabla)
        purchasing[tabla] = df
        
    return purchasing

def extractSales(conection):
    tablas = [
        "CountryRegionCurrency", "CreditCard", "Currency", "CurrencyRate", "Customer",
        "PersonCreditCard", "SalesOrderDetail", "SalesOrderHeader",
        "SalesOrderHeaderSalesReason", "SalesPerson",
        "SalesPersonQuotaHistory", "SalesReason", "SalesTaxRate",
        "SalesTerritory", "SalesTerritoryHistory", "ShoppingCartItem",
        "SpecialOffer", "SpecialOfferProduct", "Store"
    ]
    sales = {}
    for tabla in tablas:
        df = cargaSegura(conection, "Sales", tabla)
        sales[tabla] = df
        
    return sales

In [14]:
def extractEmployeeHierarchy(engine):
    query = """
    SELECT 
        e.BusinessEntityID AS EmployeeID,
        e.NationalIDNumber AS EmployeeNationalIDAlternateKey,
        e.OrganizationNode.ToString() AS OrgNode,
        m.BusinessEntityID AS ParentEmployeeKey,
        m.NationalIDNumber AS ParentEmployeeNationalIDAlternateKey
    FROM HumanResources.Employee e
    LEFT JOIN HumanResources.Employee m
        ON e.OrganizationNode.GetAncestor(1) = m.OrganizationNode;
    """
    return pd.read_sql_query(query, con=engine)

In [ ]:
humanResources =  extractHumanResources(oltp)
person = extractPerson(oltp)
production = extractProduction(oltp)
purchasing = extractPurchasing(oltp) #Funciona
sales = extractSales(oltp) #Funciona

C:\Users\lenovo\AppData\Local\Temp\ipykernel_11756\3610989036.py:5: SAWarning: Did not recognize type 'hierarchyid' of column 'OrganizationNode'
  columnas = [col["name"] for col in inspector.get_columns(table, schema=schema)]
c:\Users\lenovo\OneDrive\Desktop\Univalle\7 semestre\ciencias de datos\ETL-proyecto\my_env\Lib\site-packages\pandas\io\sql.py:1725: SAWarning: Did not recognize type 'hierarchyid' of column 'OrganizationNode'
  self.meta.reflect(bind=self.con, only=[table_name], views=True)
c:\Users\lenovo\OneDrive\Desktop\Univalle\7 semestre\ciencias de datos\ETL-proyecto\my_env\Lib\site-packages\pandas\io\sql.py:1725: SAWarning: Did not recognize type 'hierarchyid' of column 'OrganizationNode'
  self.meta.reflect(bind=self.con, only=[table_name], views=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_11756\3610989036.py:5: SAWarning: Did not recognize type 'geography' of column 'SpatialLocation'
  columnas = [col["name"] for col in inspector.get_columns(table, schema=schema)]

In [5]:
sales["Currency"].describe(include='all')

,CurrencyCode,Name,ModifiedDate
count,105,105,105
unique,105,105,NaN
top,AED,Emirati Dirham,NaN
freq,1,1,NaN
mean,NaN,NaN,2008-04-30 00:00:00
min,NaN,NaN,2008-04-30 00:00:00
25%,NaN,NaN,2008-04-30 00:00:00
50%,NaN,NaN,2008-04-30 00:00:00
75%,NaN,NaN,2008-04-30 00:00:00
max,NaN,NaN,2008-04-30 00:00:00


In [6]:
def transformDimCurrency(currency):
    dimCurrency = pd.DataFrame(columns=[
        "CurrencyKey", "CurrencyAlternateKey", "CurrencyName"
    ])
    
    dimCurrency["CurrencyAlternateKey"] = currency["CurrencyCode"] 
    dimCurrency["CurrencyName"] = currency["Name"] 
    dimCurrency["CurrencyKey"] = range(1, len(dimCurrency) + 1)
    
    return dimCurrency

In [7]:
dimCurrency = transformDimCurrency(sales["Currency"])
dimCurrency.head()

,CurrencyKey,CurrencyAlternateKey,CurrencyName
0,1,AED,Emirati Dirham
1,2,AFA,Afghani
2,3,ALL,Lek
3,4,AMD,Armenian Dram
4,5,ANG,Netherlands Antillian Guilder


In [15]:
hierarchy = extractEmployeeHierarchy(oltp)
hierarchy.head()

,EmployeeID,EmployeeNationalIDAlternateKey,OrgNode,ParentEmployeeKey,ParentEmployeeNationalIDAlternateKey
0,1,295847284,None,NaN,None
1,2,245797967,/1/,NaN,None
2,3,509647174,/1/1/,2.0,245797967
3,4,112457891,/1/1/1/,3.0,509647174
4,5,695256908,/1/1/2/,3.0,509647174


In [44]:
def transformDimEmployee(employee, employeePayHistory, employeeDepartmentHistory, department, salesPerson, person, emailAddress, personPhone, hierarchy):
    dimEmployee = pd.DataFrame(columns=[
        "EmployeeKey", "EmployeeNationalIDAlternateKey", "Title", "HireDate", "BirthDate", "LoginID",
        "MaritalStatus", "SalariedFlag", "Gender",
        "VacationHours", "SickLeaveHours", "CurrentFlag", "SalesPersonFlag", "Status"
    ])

    dimEmployee["EmployeeKey"] = employee["BusinessEntityID"]
    dimEmployee["EmployeeNationalIDAlternateKey"] = employee["NationalIDNumber"]
    dimEmployee["Title"] = employee["JobTitle"]
    dimEmployee["HireDate"] = employee["HireDate"]
    dimEmployee["BirthDate"] = employee["BirthDate"]
    dimEmployee["LoginID"] = employee["LoginID"]
    dimEmployee["MaritalStatus"] = employee["MaritalStatus"]
    dimEmployee["SalariedFlag"] = employee["SalariedFlag"].astype(int)
    dimEmployee["Gender"] = employee["Gender"]
    dimEmployee["VacationHours"] = employee["VacationHours"]
    dimEmployee["SickLeaveHours"] = employee["SickLeaveHours"]
    dimEmployee["CurrentFlag"] = employee["CurrentFlag"].astype(int)

    dimEmployee = dimEmployee.merge(
        hierarchy[["EmployeeID", "ParentEmployeeKey", "ParentEmployeeNationalIDAlternateKey"]],
        left_on="EmployeeKey",
        right_on="EmployeeID",
        how="left"
    ).drop(columns=["EmployeeID"])

    dimEmployee = dimEmployee.merge(
        salesPerson[["BusinessEntityID", "TerritoryID"]],
        left_on="EmployeeKey",
        right_on="BusinessEntityID",
        how="left"
    ).drop(columns=["BusinessEntityID"]) \
     .rename(columns={"TerritoryID": "SalesTerritoryKey"})
    
    dimEmployee["SalesTerritoryKey"] = dimEmployee["SalesTerritoryKey"].fillna(11)

    dimEmployee = dimEmployee.merge(
        person[["BusinessEntityID", "FirstName", "LastName", "MiddleName", "NameStyle"]],
        left_on="EmployeeKey",
        right_on="BusinessEntityID",
        how="left"
    ).drop(columns=["BusinessEntityID"])

    dimEmployee["NameStyle"] = dimEmployee["NameStyle"].astype(int)

    dimEmployee = dimEmployee.merge(
        emailAddress[["BusinessEntityID", "EmailAddress"]],
        left_on="EmployeeKey",
        right_on="BusinessEntityID",
        how="left"
    ).drop(columns=["BusinessEntityID"])

    dimEmployee = dimEmployee.merge( 
        personPhone[["BusinessEntityID", "PhoneNumber"]], 
        left_on="EmployeeKey", 
        right_on="BusinessEntityID", 
        how="left" 
    ).drop(columns=["BusinessEntityID"]) \
     .rename(columns={"PhoneNumber": "Phone"})
    
    dimEmployee = dimEmployee.merge(
        employeePayHistory[["BusinessEntityID", "PayFrequency", "Rate"]],
        left_on="EmployeeKey",
        right_on="BusinessEntityID",
        how="left"
    ).drop(columns=["BusinessEntityID"]) \
     .rename(columns={"Rate": "BaseRate"})

    
    dimEmployee = dimEmployee.merge(
        employeeDepartmentHistory[["BusinessEntityID", "DepartmentID"]],
        left_on="EmployeeKey",
        right_on="BusinessEntityID",
        how="left"
    ).merge(
        department[["DepartmentID", "Name"]],
        on="DepartmentID",
        how="left"
    ).rename(columns={"Name": "DepartmentName"}).drop(columns=["BusinessEntityID", "DepartmentID"])


    dimEmployee["SalesPersonFlag"] = np.where(
        (dimEmployee["DepartmentName"].str.contains("Sales", na=False)) &
        (dimEmployee["Title"] != "Vice President of Engineering"),
        1,
        0
    )

    dimEmployee = dimEmployee.merge(
        employeeDepartmentHistory[["BusinessEntityID", "StartDate", "EndDate"]],
        left_on="EmployeeKey",
        right_on="BusinessEntityID",
        how="left"
    ).drop(columns=["BusinessEntityID"])

    dimEmployee["Status"] = np.where(
        dimEmployee["EndDate"].isna(),
        "Current",
        None
    )

    column_order = [
        "EmployeeKey", "ParentEmployeeKey", "EmployeeNationalIDAlternateKey", "ParentEmployeeNationalIDAlternateKey", 
        "SalesTerritoryKey", "FirstName", "LastName", "MiddleName", "NameStyle", "Title", "HireDate", "BirthDate", 
        "LoginID", "EmailAddress", "Phone", "MaritalStatus", "SalariedFlag", "Gender", "PayFrequency", "BaseRate", 
        "VacationHours", "SickLeaveHours", "CurrentFlag", "SalesPersonFlag", "DepartmentName", "StartDate", "EndDate","Status"
    ]

    dimEmployee = dimEmployee[column_order]
    dimEmployee = dimEmployee.drop_duplicates(subset=["EmployeeKey"])

    
    return dimEmployee

In [45]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

In [46]:
transformDimEmployee(
    humanResources["Employee"],
    humanResources["EmployeePayHistory"],
    humanResources["EmployeeDepartmentHistory"],
    humanResources["Department"],
    sales["SalesPerson"],
    person["Person"],
    person["EmailAddress"],
    person["PersonPhone"],
    hierarchy
).head(10)

,EmployeeKey,ParentEmployeeKey,EmployeeNationalIDAlternateKey,ParentEmployeeNationalIDAlternateKey,SalesTerritoryKey,FirstName,LastName,MiddleName,NameStyle,Title,HireDate,BirthDate,LoginID,EmailAddress,Phone,MaritalStatus,SalariedFlag,Gender,PayFrequency,BaseRate,VacationHours,SickLeaveHours,CurrentFlag,SalesPersonFlag,DepartmentName,StartDate,EndDate,Status
0,1,NaN,295847284,None,11.0,Ken,Sánchez,J,0,Chief Executive Officer,2009-01-14,1969-01-29,adventure-works\ken0,ken0@adventure-works.com,697-555-0142,S,1,M,2,125.5000,99,69,1,0,Executive,2009-01-14,NaT,Current
1,2,NaN,245797967,None,11.0,Terri,Duffy,Lee,0,Vice President of Engineering,2008-01-31,1971-08-01,adventure-works\terri0,terri0@adventure-works.com,819-555-0175,S,1,F,2,63.4615,1,20,1,0,Engineering,2008-01-31,NaT,Current
2,3,2.0,509647174,245797967,11.0,Roberto,Tamburello,None,0,Engineering Manager,2007-11-11,1974-11-12,adventure-works\roberto0,roberto0@adventure-works.com,212-555-0187,M,1,M,2,43.2692,2,21,1,0,Engineering,2007-11-11,NaT,Current
3,4,3.0,112457891,509647174,11.0,Rob,Walters,None,0,Senior Tool Designer,2007-12-05,1974-12-23,adventure-works\rob0,rob0@adventure-works.com,612-555-0100,S,0,M,2,8.6200,48,80,1,0,Engineering,2007-12-05,2010-05-30,None
15,5,3.0,695256908,509647174,11.0,Gail,Erickson,A,0,Design Engineer,2008-01-06,1952-09-27,adventure-works\gail0,gail0@adventure-works.com,849-555-0139,M,1,F,2,32.6923,5,22,1,0,Engineering,2008-01-06,NaT,Current
16,6,3.0,998320692,509647174,11.0,Jossef,Goldberg,H,0,Design Engineer,2008-01-24,1959-03-11,adventure-works\jossef0,jossef0@adventure-works.com,122-555-0189,M,1,M,2,32.6923,6,23,1,0,Engineering,2008-01-24,NaT,Current
17,7,3.0,134969118,509647174,11.0,Dylan,Miller,A,0,Research and Development Manager,2009-02-08,1987-02-24,adventure-works\dylan0,dylan0@adventure-works.com,181-555-0156,M,1,M,2,50.4808,61,50,1,0,Research and Development,2009-02-08,NaT,Current
18,8,7.0,811994146,134969118,11.0,Diane,Margheim,L,0,Research and Development Engineer,2008-12-29,1986-06-05,adventure-works\diane1,diane1@adventure-works.com,815-555-0138,S,1,F,2,40.8654,62,51,1,0,Research and Development,2008-12-29,NaT,Current
19,9,7.0,658797903,134969118,11.0,Gigi,Matthew,N,0,Research and Development Engineer,2009-01-16,1979-01-21,adventure-works\gigi0,gigi0@adventure-works.com,185-555-0186,M,1,F,2,40.8654,63,51,1,0,Research and Development,2009-01-16,NaT,Current
20,10,7.0,879342154,134969118,11.0,Michael,Raheem,None,0,Research and Development Manager,2009-05-03,1984-11-30,adventure-works\michael6,michael6@adventure-works.com,330-555-2568,M,1,M,2,42.4808,16,64,1,0,Research and Development,2009-05-03,NaT,Current


In [79]:
def transformDimSalesTerritory(SalesTerritory):
    dimSalesTerritory = pd.DataFrame(columns=[
        "SalesTerritoryKey", "SalesTerritoryAlternateKey", "SalesTerritoryRegion", "SalesTerritoryCountry", "SalesTerritoryGroup"
    ])

    dimSalesTerritory["SalesTerritoryKey"] = SalesTerritory["TerritoryID"]
    dimSalesTerritory["SalesTerritoryRegion"] = SalesTerritory["Name"]
    dimSalesTerritory["SalesTerritoryCountry"] = SalesTerritory["CountryRegionCode"]
    dimSalesTerritory["SalesTerritoryGroup"] = SalesTerritory["Group"]

    dimSalesTerritory.loc[len(dimSalesTerritory)] = [
        11,            # SalesTerritoryKey
        0,          # SalesTerritoryAlternateKey
        "NA",       # SalesTerritoryRegion
        "NA",         # SalesTerritoryCountry
        "NA"        # SalesTerritoryGroup
    ]


    return dimSalesTerritory

In [80]:
transformDimSalesTerritory(sales["SalesTerritory"])

,SalesTerritoryKey,SalesTerritoryAlternateKey,SalesTerritoryRegion,SalesTerritoryCountry,SalesTerritoryGroup
0,1,NaN,Northwest,US,North America
1,2,NaN,Northeast,US,North America
2,3,NaN,Central,US,North America
3,4,NaN,Southwest,US,North America
4,5,NaN,Southeast,US,North America
5,6,NaN,Canada,CA,North America
6,7,NaN,France,FR,Europe
7,8,NaN,Germany,DE,Europe
8,9,NaN,Australia,AU,Pacific
9,10,NaN,United Kingdom,GB,Europe
